In [4]:
# Import the libraries we use.
from bs4 import BeautifulSoup as bsoup
import datetime
import glob
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time
import urllib.request
from urllib.request import urlopen
import zipfile

# Save the initial working directory.
start_directory = os.getcwd()
print(start_directory)

# Create the path to the data directory.
data_directory = os.path.join('.', 'data_rejected')
print(data_directory)

C:\Users\ankur
.\data_rejected


In [7]:
# Create logfile.
logfile = open("time_data.txt", "w") 
def log_entry(s):
    #print('Date now: %s' % datetime.datetime.now())

    timestamp = '[%s] ' % datetime.datetime.now()
    log_line = timestamp + s + '\n'
    logfile.write(log_line)
    logfile.flush()
    
    # Also write to standard output as a convenience.
    print(log_line)

In [6]:
base_URL = "https://resources.lendingclub.com"
url = urllib.request.urlopen("https://www.lendingclub.com/info/download-data.action")
content = url.read()
#print(content)

soup = bsoup(content, 'html')
#print(soup) 

#loanStatsFileNamesJS
fileNameDiv = soup.find('div', {"id":"rejectedLoanStatsFileNamesJS"})
loanFileList = fileNameDiv.text.rstrip("|")
print(loanFileList)

# Set the data directory as the current working directory for the downloads.
os.chdir(r'C:/Users/ankur/data')

# Download and extract all the data files.
for fileName in loanFileList.split("|"):
    print(fileName)
    
    csv_filename = fileName
    if csv_filename.endswith('.zip'):
        csv_filename = csv_filename[:-4]
        
    csv_filepath = os.path.join('.', csv_filename)
    #print(csv_filepath)
    
    # Download the file if it isn't already in our data directory.
    if os.path.exists(csv_filepath):
        print("Already downloaded %s" % csv_filepath)
    else:
        print("Downloading file %s" % csv_filepath)
        file_URL = base_URL + '/' + fileName
        #print(file_URL)
        
        zfile = requests.get(file_URL)
        z = zipfile.ZipFile(io.BytesIO(zfile.content))
        z.extractall()

# Restore the working directory
os.chdir(start_directory)

C:\Users\ankur\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\ankur\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


RejectStatsA.csv.zip|RejectStatsB.csv.zip|RejectStatsD.csv.zip|RejectStats_2016Q1.csv.zip|RejectStats_2016Q2.csv.zip|RejectStats_2016Q3.csv.zip|RejectStats_2016Q4.csv.zip
RejectStatsA.csv.zip
RejectStatsB.csv.zip
RejectStatsD.csv.zip
RejectStats_2016Q1.csv.zip
RejectStats_2016Q2.csv.zip
RejectStats_2016Q3.csv.zip
RejectStats_2016Q4.csv.zip


In [9]:
sample_filepath = os.path.join(r'C:/Users/Ankur/data_rejected', 'RejectStats_2016Q4.csv')
data = pd.read_csv(sample_filepath, skiprows=1)
data.head()
data.shape

(1404490, 9)

In [10]:
def concatenate(indir="./", outfilename="./Rejected_LoanData.csv"):
    initial_working_dir = os.getcwd()
    os.chdir(indir)
    
    csvFileList = glob.glob("*.csv")
    dfList = []
   
    # Process the CSV files, without the initial line.
    for csv_filename in csvFileList:
        print(csv_filename)
        
        # Use the file modification time to track when the data was downloaded.
        timestamp = int(os.path.getmtime(csv_filename))
        #print("last modified: %s" % str(timestamp))
        
        df = pd.read_csv(csv_filename, low_memory=False, skiprows=1)
        
        # Add the timestamp into the data.
        df['timestamp'] = timestamp
        print(df.shape)
        
        dfList.append(df)
        
    concatDf = pd.concat(dfList, axis=0, copy=False)
    #concatDf.columns = columns
    concatDf.to_csv(outfilename, index=None)
    print(concatDf.shape)
    
    # Restore the working directory.
    os.chdir(initial_working_dir)

In [11]:
concatenate(indir=r'C:/Users/Ankur/data_rejected')

RejectStatsA.csv
(755491, 10)
RejectStatsB.csv
(2694642, 10)
RejectStatsD.csv
(2859379, 10)
RejectStats_2016Q1.csv
(1096204, 10)
RejectStats_2016Q2.csv
(996561, 10)
RejectStats_2016Q3.csv
(1272619, 10)
RejectStats_2016Q4.csv
(1404490, 10)
(11079386, 10)
